**Objetivo:** Limpar e preparar os dados.

**Conteúdo:**

*   Carregar o seu dataset sintético.
*   Realizar uma análise exploratória de dados (EDA) para entender a distribuição dos dados e identificar possíveis problemas.
* Descrever os passos para transformar os dados brutos em um formato adequado para o seu pipeline.
* Salvar o dataset pré-processado em um novo arquivo para uso nos próximos notebooks.